<a href="https://colab.research.google.com/github/DavidSchneider47/ALIrecommendation/blob/main/Amenities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install overpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 964.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for overpy: filename=overpy-0.6-py3-none-any.whl size=14091 sha256=5aef74875295693a32527ec8c5d859888e7c59687874a610357500e912fa5772
  Stored in directory: /root/.cache/pip/wheels/6f/05/ef/529f51b5983091d5b53077e4f2f81d052a939573c5dd07acfc
Successfully built overpy


In [2]:
import overpy

def query_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()

    # Begin the Overpass QL query
    query = f"[out:json];("

    # Loop through each tag and add it to the query
    for tag in tags:
        key, value = tag.split("=")
        query += f"""
        node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        """

    # Close the query
    query += ");out body;>;out skel qt;"

    # Execute the query
    result = api.query(query)

    # Print the results
    for node in result.nodes:
        print(f"{node.tags.get(key)} node at lat={node.lat}, lon={node.lon}")
    for way in result.ways:
        print(f"{way.tags.get(key)} way with id={way.id}")

# Coordinates, radius, and list of tags
lat = 38.9844
lon = -77.0958
radius = 402.34  # 1/4 mile in meters
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

query_amenities_nearby(lat, lon, radius, tags_list)


None node at lat=38.9872748, lon=-77.0945738
None node at lat=38.9815850, lon=-77.0953003
None node at lat=38.9851278, lon=-77.0924343
clothes node at lat=38.9810804, lon=-77.0967328
clothes node at lat=38.9813145, lon=-77.0959096
clothes node at lat=38.9812107, lon=-77.0979540
clothes node at lat=38.9818258, lon=-77.0977274
clothes node at lat=38.9816496, lon=-77.0979621
clothes node at lat=38.9815741, lon=-77.0979560
clothes node at lat=38.9811683, lon=-77.0958482
clothes node at lat=38.9832783, lon=-77.0934261
clothes node at lat=38.9815770, lon=-77.0931039
clothes node at lat=38.9822273, lon=-77.0973280
None node at lat=38.9815102, lon=-77.0993779
None node at lat=38.9808934, lon=-77.0991782
None node at lat=38.9809991, lon=-77.0985760
None node at lat=38.9812051, lon=-77.0992000
None node at lat=38.9808985, lon=-77.0990587
None node at lat=38.9815426, lon=-77.0986141
None node at lat=38.9811944, lon=-77.0994519
None node at lat=38.9813165, lon=-77.0994605
None node at lat=38.98132

In [3]:
import overpy

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Coordinates, radius, and list of tags
lat = 38.9844
lon = -77.0958
radius = 402.34  # 1/4 mile in meters
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

results = check_amenities_nearby(lat, lon, radius, tags_list)

# Print the results
for tag, presence in results.items():
    print(f"{tag}: {presence}")


amenity=community_centre: 0
amenity=library: 1
tourism=museum: 0
amenity=theatre: 1
amenity=cinema: 1
amenity=nightclub: 0
leisure=bowling_alley: 0
amenity=pharmacy: 0
amenity=hospital: 0
amenity=clinic: 0
amenity=doctors: 0
amenity=dentist: 0
shop=supermarket: 1
shop=clothes: 1
shop=fashion: 0


In [7]:
import overpy
import csv

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Load latitudes and longitudes from the provided CSV
coordinates = []
with open('Coordinates.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    coordinates = [(float(row[0]), float(row[1])) for row in csvreader]  # Assuming latitude is in the first column and longitude in the second

# Define the radius and tags list
radius = 402.34
tags_list = [
    # [Your list of tags]
]

# Open an output CSV to save the results
with open('CoordinatesFormatted.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    headers = ["Latitude", "Longitude"] + tags_list
    csvwriter.writerow(headers)

    # Loop through each coordinate pair and fetch results
    for lat, lon in coordinates:
        results = check_amenities_nearby(lat, lon, radius, tags_list)
        row_data = [lat, lon] + [results[tag] for tag in tags_list]
        csvwriter.writerow(row_data)

print("CoordinatesFormatted.csv")


CoordinatesFormatted.csv


In [8]:
from google.colab import files

files.download('CoordinatesFormatted.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import overpy
import csv

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Load latitudes and longitudes from the provided CSV
coordinates = []
with open('LatLon5.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    coordinates = [(float(row[0]), float(row[1])) for row in csvreader]  # Assuming latitude is in the first column and longitude in the second

# Define the radius and tags list
radius = 402.34
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

# Open an output CSV to save the results
with open('LatLon5FormattedResults.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    headers = ["Latitude", "Longitude"] + tags_list
    csvwriter.writerow(headers)

    # Loop through each coordinate pair and fetch results
    for lat, lon in coordinates:
        results = check_amenities_nearby(lat, lon, radius, tags_list)
        row_data = [lat, lon] + [results[tag] for tag in tags_list]
        csvwriter.writerow(row_data)

print("Results saved to LatLon5FormattedResults.csv")


Results saved to LatLon5FormattedResults.csv


In [ ]:
import overpy
import csv

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Load latitudes and longitudes from the provided CSV
coordinates = []
with open('Coordinates.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    coordinates = [(float(row[1]), float(row[0])) for row in csvreader]  # Assuming latitude is in the first column and longitude in the second

radius = 402.34  # 1/4 mile in meters
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

# Check if the output CSV exists to decide on writing headers
file_exists = False
try:
    with open('osm_amenities_results.csv', 'r') as f:
        file_exists = True
except FileNotFoundError:
    pass

with open('osm_amenities_results.csv', 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row only if the file didn't exist
    if not file_exists:
        headers = ["lat", "lon"] + tags_list
        csvwriter.writerow(headers)

    # Loop through each coordinate pair and fetch results
    for lat, lon in coordinates:
        results = check_amenities_nearby(lat, lon, radius, tags_list)
        row_data = [lat, lon] + [results[tag] for tag in tags_list]
        csvwriter.writerow(row_data)

print("Results appended to osm_amenities_results1.csv")


In [15]:
from google.colab import files
files.download('osm_amenities_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import files

files.download('osm_amenities_results1.csv')


FileNotFoundError: ignored

In [ ]:
import overpy
import csv

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Coordinates, radius, and list of tags
lat = 38.9844
lon = -77.0958
radius = 402.34  # 1/4 mile in meters
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

results = check_amenities_nearby(lat, lon, radius, tags_list)

# Export the results to a CSV file
with open('osm_amenities_results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    headers = ["Latitude", "Longitude"] + tags_list
    csvwriter.writerow(headers)

    # Write the results row
    row_data = [lat, lon] + [results[tag] for tag in tags_list]
    csvwriter.writerow(row_data)

print("Results saved to osm_amenities_results.csv")


Results saved to osm_amenities_results.csv


In [10]:
import overpy
import csv
import time
time.sleep(2)  # Introduces a delay of 2 seconds between requests

def check_amenities_nearby(lat, lon, radius_meters, tags):
    api = overpy.Overpass()
    results_dict = {}

    for tag in tags:
        key, value = tag.split("=")
        query = f"""
        [out:json];
        (
            node["{key}"="{value}"](around:{radius_meters},{lat},{lon});
            way["{key}"="{value}"](around:{radius_meters},{lat},{lon});
        );
        out body;
        """

        # Execute the query
        result = api.query(query)

        # Check the presence of the amenity/tag
        if len(result.nodes) + len(result.ways) > 0:
            results_dict[tag] = 1
        else:
            results_dict[tag] = 0

    return results_dict

# Load latitudes and longitudes from the provided CSV
coordinates = []
with open('Coordinates.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    coordinates = [(float(row[1]), float(row[0])) for row in csvreader]  # Assuming latitude is in the first column and longitude in the second

radius = 402.34  # 1/4 mile in meters
tags_list = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

# Check if the output CSV exists to decide on writing headers
file_exists = False
try:
    with open('osm_amenities_results2.csv', 'r') as f:
        file_exists = True
except FileNotFoundError:
    pass

with open('osm_amenities_results.csv2', 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row only if the file didn't exist
    if not file_exists:
        headers = ["Latitude", "Longitude"] + tags_list
        csvwriter.writerow(headers)

    # Loop through each coordinate pair and fetch results
    for lat, lon in coordinates:
        results = check_amenities_nearby(lat, lon, radius, tags_list)
        row_data = [lat, lon] + [results[tag] for tag in tags_list]
        csvwriter.writerow(row_data)

print("Results appended to osm_amenities_results2.csv")


Results appended to osm_amenities_results2.csv


In [11]:
from google.colab import files

files.download('osm_amenities_results2.csv')


FileNotFoundError: ignored

In [ ]:
import csv
import overpy

def query_amenities_nearby(lat, lon, radius_meters, amenities):
    api = overpy.Overpass()

    results = {}
    for amenity in amenities:
        # Build the Overpass QL query
        query = f"""
        [out:json];
        (
          node["{amenity}"](around:{radius_meters},{lat},{lon});
          way["{amenity}"](around:{radius_meters},{lat},{lon});
        );
        out count;
        """

        # Execute the query and store the count
        try:
            result = api.query(query)
            # Check if there are nodes or ways returned for the amenity
            count = len(result.nodes) + len(result.ways)
            results[amenity] = 1 if count > 0 else 0
        except Exception as e:
            print(f"Error querying for {amenity} at lat={lat}, lon={lon}: {e}")
            results[amenity] = 0

    return results

# Load coordinates from CSV
with open('LatLon3.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    coordinates = [(float(row[1]), float(row[0])) for row in csvreader]

radius = 402.34  # 1/4 mile in meters
amenities = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

# Open the output CSV
with open('osm_amenities_results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    headers = ["Latitude", "Longitude"] + amenities
    csvwriter.writerow(headers)

    # Loop through coordinates and fetch results
    for lat, lon in coordinates:
        print(f"Querying for lat={lat}, lon={lon}")

        results_dict = query_amenities_nearby(lat, lon, radius, amenities)

        print(f"Results for lat={lat}, lon={lon}: {results_dict}")

        # Write results to the CSV
        row_data = [lat, lon] + [results_dict[amenity] for amenity in amenities]
        csvwriter.writerow(row_data)

print("Results saved to osm_amenities_results1.csv")


Querying for lat=-76.9082, lon=39.01324
Results for lat=-76.9082, lon=39.01324: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 0, 'amenity=theatre': 0, 'amenity=cinema': 0, 'amenity=nightclub': 0, 'leisure=bowling_alley': 0, 'amenity=pharmacy': 0, 'amenity=hospital': 0, 'amenity=clinic': 0, 'amenity=doctors': 0, 'amenity=dentist': 0, 'shop=supermarket': 0, 'shop=clothes': 0, 'shop=fashion': 0}
Querying for lat=-77.02253, lon=38.89457
Results for lat=-77.02253, lon=38.89457: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 0, 'amenity=theatre': 0, 'amenity=cinema': 0, 'amenity=nightclub': 0, 'leisure=bowling_alley': 0, 'amenity=pharmacy': 0, 'amenity=hospital': 0, 'amenity=clinic': 0, 'amenity=doctors': 0, 'amenity=dentist': 0, 'shop=supermarket': 0, 'shop=clothes': 0, 'shop=fashion': 0}
Querying for lat=-77.03238, lon=38.90101
Results for lat=-77.03238, lon=38.90101: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 

In [ ]:
import csv
import overpy

def query_amenities_nearby(lat, lon, radius_meters, amenities):
    api = overpy.Overpass()

    results = {}
    for amenity in amenities:
        # Build the Overpass QL query
        query = f"""
        [out:json];
        (
          node["{amenity}"](around:{radius_meters},{lat},{lon});
          way["{amenity}"](around:{radius_meters},{lat},{lon});
        );
        out count;
        """

        # Execute the query and store the count
        try:
            result = api.query(query)
            # Check if there are nodes or ways returned for the amenity
            count = len(result.nodes) + len(result.ways)
            results[amenity] = 1 if count > 0 else 0
        except Exception as e:
            print(f"Error querying for {amenity} at lat={lat}, lon={lon}: {e}")
            results[amenity] = 0

    return results

# Load coordinates from CSV
with open('LatLon3.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    corrected_coordinates = [(float(row[0]), float(row[1])) for row in csvreader]

radius = 402.34  # 1/4 mile in meters
amenities = [
    "amenity=community_centre",
    "amenity=library",
    "tourism=museum",
    "amenity=theatre",
    "amenity=cinema",
    "amenity=nightclub",
    "leisure=bowling_alley",
    "amenity=pharmacy",
    "amenity=hospital",
    "amenity=clinic",
    "amenity=doctors",
    "amenity=dentist",
    "shop=supermarket",
    "shop=clothes",
    "shop=fashion"
]

# Open the output CSV
with open('osm_amenities_results1.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    headers = ["Latitude", "Longitude"] + amenities
    csvwriter.writerow(headers)

    # Loop through coordinates and fetch results
    for lat, lon in corrected_coordinates:
        print(f"Querying for lat={lat}, lon={lon}")

        results_dict = query_amenities_nearby(lat, lon, radius, amenities)

        print(f"Results for lat={lat}, lon={lon}: {results_dict}")

        # Write results to the CSV
        row_data = [lat, lon] + [results_dict[amenity] for amenity in amenities]
        csvwriter.writerow(row_data)

print("Results saved to osm_amenities_results1.csv")


Querying for lat=39.01324, lon=-76.9082
Results for lat=39.01324, lon=-76.9082: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 0, 'amenity=theatre': 0, 'amenity=cinema': 0, 'amenity=nightclub': 0, 'leisure=bowling_alley': 0, 'amenity=pharmacy': 0, 'amenity=hospital': 0, 'amenity=clinic': 0, 'amenity=doctors': 0, 'amenity=dentist': 0, 'shop=supermarket': 0, 'shop=clothes': 0, 'shop=fashion': 0}
Querying for lat=38.89457, lon=-77.02253
Results for lat=38.89457, lon=-77.02253: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 0, 'amenity=theatre': 0, 'amenity=cinema': 0, 'amenity=nightclub': 0, 'leisure=bowling_alley': 0, 'amenity=pharmacy': 0, 'amenity=hospital': 0, 'amenity=clinic': 0, 'amenity=doctors': 0, 'amenity=dentist': 0, 'shop=supermarket': 0, 'shop=clothes': 0, 'shop=fashion': 0}
Querying for lat=38.90101, lon=-77.03238
Results for lat=38.90101, lon=-77.03238: {'amenity=community_centre': 0, 'amenity=library': 0, 'tourism=museum': 